In [35]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [36]:
from Utils.recsys2022DataReader import createBumpURM
from Utils.recsys2022DataReader import createSmallICM
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Hybrid.MergedHybridRecommender import MergedHybridRecommender
from optuna.samplers import TPESampler
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
import optuna as op

In [37]:
URM = createBumpURM()
ICM = createSmallICM()

C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Utils\recsys2022DataReader.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(urmPath)


In [38]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, 0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 952 ( 2.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1501 ( 3.6%) Users that have less than 1 test interactions


In [39]:
ItemKNNCF_recommender = ItemKNNCFRecommender(URM_train)
ItemKNNCBF_recommender = ItemKNNCBFRecommender(URM_train, ICM)

ItemKNNCBFRecommender: ICM Detected 4877 (19.9%) items with no features.


In [40]:
def objective(trial):

    topKCF = trial.suggest_int("topKCF", 300, 400)
    shrinkCF = trial.suggest_float("shrinkCF", 10, 100)
    topKCBF = trial.suggest_int("topKCBF", 300, 400)
    shrinkCBF = trial.suggest_float("shrinkCBF", 10, 100)
    alpha = trial.suggest_float("alpha", 0.1, 0.9)

    ItemKNNCF_recommender.fit(topK=topKCF, shrink=shrinkCF)
    ItemKNNCBF_recommender.fit(topK=topKCBF, shrink=shrinkCBF)

    recommender = MergedHybridRecommender(URM_train, recommender1=ItemKNNCF_recommender, recommender2=ItemKNNCBF_recommender, verbose=False)
    recommender.fit(alpha=alpha)

    result, _ = evaluator_validation.evaluateRecommender(recommender)

    return result.loc[10]['MAP']

In [41]:
study = op.create_study(direction='maximize', sampler=TPESampler(multivariate=True))
study.optimize(objective, n_trials=10)

C:\Users\Mike\anaconda3\envs\recsys\lib\site-packages\optuna\samplers\_tpe\sampler.py:263: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2022-11-11 15:03:37,139] A new study created in memory with name: no-name-bcf0ed3d-8833-45bd-8bc8-28ba15cd11b7


Similarity column 24507 (100.0%), 6582.15 column/sec. Elapsed time 3.72 sec
Compute_Similarity: detected dense matrix
Similarity column 24507 (100.0%), 1654.73 column/sec. Elapsed time 14.81 sec
EvaluatorHoldout: Processed 40128 (100.0%) in 41.33 sec. Users per second: 971


[I 2022-11-11 15:04:37,528] Trial 0 finished with value: 0.013244140575555455 and parameters: {'topKCF': 367, 'shrinkCF': 34.9206687128155, 'topKCBF': 358, 'shrinkCBF': 84.22327444074578, 'alpha': 0.20881971158487833}. Best is trial 0 with value: 0.013244140575555455.


Similarity column 24507 (100.0%), 6629.22 column/sec. Elapsed time 3.70 sec
Compute_Similarity: detected dense matrix
Similarity column 24507 (100.0%), 1461.99 column/sec. Elapsed time 16.76 sec
EvaluatorHoldout: Processed 40128 (100.0%) in 41.93 sec. Users per second: 957


[I 2022-11-11 15:05:40,405] Trial 1 finished with value: 0.011769793011569723 and parameters: {'topKCF': 302, 'shrinkCF': 71.59387529364699, 'topKCBF': 308, 'shrinkCBF': 56.39277025166837, 'alpha': 0.1991968664447339}. Best is trial 0 with value: 0.013244140575555455.


...

[I 2022-11-11 15:11:57,251] Trial 7 finished with value: 0.004331877080643508 and parameters: {'topKCF': 346, 'shrinkCF': 40.67170264271434, 'topKCBF': 358, 'shrinkCBF': 11.946537187460892, 'alpha': 0.337648222297975}. Best is trial 6 with value: 0.01666525056327677.


Similarity column 24507 (100.0%), 6226.58 column/sec. Elapsed time 3.94 sec
Compute_Similarity: detected dense matrix
Similarity column 24507 (100.0%), 1388.63 column/sec. Elapsed time 17.65 sec
EvaluatorHoldout: Processed 40128 (100.0%) in 47.68 sec. Users per second: 842


[I 2022-11-11 15:13:07,118] Trial 8 finished with value: 0.006688281032442274 and parameters: {'topKCF': 376, 'shrinkCF': 54.67130509139248, 'topKCBF': 351, 'shrinkCBF': 11.766957412003926, 'alpha': 0.46287735178468814}. Best is trial 6 with value: 0.01666525056327677.


Similarity column 24507 (100.0%), 6223.76 column/sec. Elapsed time 3.94 sec
Compute_Similarity: detected dense matrix
Similarity column 24507 (100.0%), 1182.66 column/sec. Elapsed time 20.72 sec
EvaluatorHoldout: Processed 40128 (100.0%) in 47.90 sec. Users per second: 838


[I 2022-11-11 15:14:20,344] Trial 9 finished with value: 0.010781561503190087 and parameters: {'topKCF': 358, 'shrinkCF': 18.690411137466956, 'topKCBF': 383, 'shrinkCBF': 12.141412765331012, 'alpha': 0.625463301747489}. Best is trial 6 with value: 0.01666525056327677.


In [42]:
ItemKNNCF_recommender.fit(topK=study.best_params['topKCF'], shrink=study.best_params['shrinkCF'])
ItemKNNCBF_recommender.fit(topK=study.best_params['topKCBF'], shrink=study.best_params['shrinkCBF'])

recommender = MergedHybridRecommender(URM_train, recommender1=ItemKNNCF_recommender, recommender2=ItemKNNCBF_recommender, verbose=False)
recommender.fit(alpha=study.best_params['alpha'])

Similarity column 24507 (100.0%), 6265.55 column/sec. Elapsed time 3.91 sec
Compute_Similarity: detected dense matrix
Similarity column 24507 (100.0%), 1553.55 column/sec. Elapsed time 15.77 sec


In [43]:
result, _ = evaluator_test.evaluateRecommender(recommender)
result

EvaluatorHoldout: Processed 40677 (100.0%) in 44.77 sec. Users per second: 909


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.041707,0.069817,0.065284,0.019218,0.031946,0.137631,0.071077,0.050898,0.294663,0.161713,...,0.977131,0.287924,0.977131,0.069519,9.34443,0.992565,0.13737,0.692641,3.949131,0.301382
